In [1]:
from eda_report import get_word_report

## 1. Load a dataset

The program expects a pandas `DataFrame`.

In [2]:
# Using the auto-mpg sample dataset
import seaborn as sns
data = sns.load_dataset('mpg')
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


## 2. Generate the report

You can customize the following:

- `title`: default = 'Exploratory Data Analysis Report',
- `graph_colour`: default = 'orangered',
- `output_filename`: default = 'basic-report.docx'

In [3]:
get_word_report(data)

[INFO 18:59:37.593] Assessing correlation in numeric variables...
[INFO 18:59:49.840] Done. Summarising each variable...
[INFO 18:59:51.449] Done. Results saved as 'basic-report.docx'
